In [1]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader


In [ ]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv"
)
df.head()


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [ ]:
df.drop(columns=["id", "Unnamed: 32"], inplace=True)
df.head()


,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [15]:
X_train, X_test, y_train, y_test = train_test_split(
    df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2
)
y_train


320    B
256    M
252    M
305    B
183    B
      ..
482    B
144    B
17     M
365    M
95     M
Name: diagnosis, Length: 455, dtype: object

In [16]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
X_train


,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
398,B,11.06,14.83,70.31,378.2,0.07741,0.04768,0.027120,0.007246,0.1535,...,20.35,80.79,496.7,0.1120,0.18790,0.20790,0.05556,0.2590,0.09158,NaN
353,M,15.08,25.74,98.00,716.6,0.10240,0.09769,0.123500,0.065530,0.1647,...,33.22,121.20,1050.0,0.1660,0.23560,0.40290,0.15260,0.2654,0.09438,NaN
466,B,13.14,20.74,85.98,536.9,0.08675,0.10890,0.108500,0.035100,0.1562,...,25.46,100.90,689.1,0.1351,0.35490,0.45040,0.11810,0.2563,0.08174,NaN
185,B,10.08,15.11,63.76,317.5,0.09267,0.04695,0.001597,0.002404,0.1703,...,21.18,75.39,437.0,0.1521,0.10190,0.00692,0.01042,0.2933,0.07697,NaN
99,M,14.42,19.77,94.48,642.5,0.09752,0.11410,0.093880,0.058390,0.1879,...,30.86,109.50,826.4,0.1431,0.30260,0.31940,0.15650,0.2718,0.09353,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,B,14.87,20.21,96.12,680.9,0.09587,0.08345,0.068240,0.049510,0.1487,...,28.48,103.90,783.6,0.1216,0.13880,0.17000,0.10170,0.2369,0.06599,NaN
414,M,15.13,29.81,96.71,719.5,0.08320,0.04605,0.046860,0.027390,0.1852,...,36.91,110.10,931.4,0.1148,0.09866,0.15470,0.06575,0.3233,0.06165,NaN
264,M,17.19,22.07,111.60,928.3,0.09726,0.08995,0.090610,0.065270,0.1867,...,29.33,140.50,1436.0,0.1558,0.25670,0.38890,0.19840,0.3216,0.07570,NaN
16,M,14.68,20.13,94.74,684.5,0.09867,0.07200,0.073950,0.052590,0.1586,...,30.88,123.40,1138.0,0.1464,0.18710,0.29140,0.16090,0.3029,0.08216,NaN


In [18]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)


In [ ]:
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train).to(torch.float64)
y_test_tensor = torch.from_numpy(y_test).to(torch.float64)


In [20]:
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self, index):
        return self.features[index], self.labels[index]


In [21]:
train_dataset = CustomDataset(X_train_tensor, y_train_tensor)
test_dataset = CustomDataset(X_test_tensor, y_test_tensor)


In [22]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)


In [23]:
class MyNNModule(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.linear = nn.Linear(num_features, 1, dtype=torch.float64)
        self.sigmoid = nn.Sigmoid()
        self.loss_function = nn.BCELoss()

    def forward(self, features):
        z = self.linear(features)
        y_pred = self.sigmoid(z)
        return y_pred

    # def loss_function(self, y_pred, y_actual):
    #     epsilon = 1e-7
    #     y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)
    #     loss = -(
    #         y_actual * torch.log(y_pred) + (1 - y_actual) * torch.log(1 - y_pred)
    #     ).mean()
    #     return loss


In [24]:
learning_rate = 0.1
epochs = 25


In [25]:
model = MyNNModule(X_train_tensor.shape[1])


In [26]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


In [ ]:
for epoch in range(epochs):
    for batch_features, batch_labels in train_dataloader:
        y_pred = model(batch_features)

        loss = model.loss_function(y_pred, batch_labels.view(-1, 1))

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()
        # y_pred = model(X_train_tensor)

        # loss = model.loss_function(y_pred, y_train_tensor.view(-1, 1))

        # optimizer.zero_grad()

        # loss.backward()

        # optimizer.step()


        print(f"Epoch: {epoch+1}, Loss: {loss}")


Epoch: 1, Loss: 0.8036232112583372
Epoch: 1, Loss: 0.5888680984239889
Epoch: 1, Loss: 0.4720330145556181
Epoch: 1, Loss: 0.45596651343772965
Epoch: 1, Loss: 0.3844729307300639
Epoch: 1, Loss: 0.3032101095852583
Epoch: 1, Loss: 0.2790923917465761
Epoch: 1, Loss: 0.2816394039751583
Epoch: 1, Loss: 0.24721690668636664
Epoch: 1, Loss: 0.3496982434463166
Epoch: 1, Loss: 0.22815076830358288
Epoch: 1, Loss: 0.22214212616196446
Epoch: 1, Loss: 0.26514942694681437
Epoch: 1, Loss: 0.23004841649817112
Epoch: 1, Loss: 0.15447625071161278
Epoch: 2, Loss: 0.24799318267170933
Epoch: 2, Loss: 0.17404321224450836
Epoch: 2, Loss: 0.23987297942585617
Epoch: 2, Loss: 0.22410268974320177
Epoch: 2, Loss: 0.1685729886791206
Epoch: 2, Loss: 0.21166968739855369
Epoch: 2, Loss: 0.17340828793879154
Epoch: 2, Loss: 0.1678067116329409
Epoch: 2, Loss: 0.12862608091943586
Epoch: 2, Loss: 0.19294365731520774
Epoch: 2, Loss: 0.17368742607027451
Epoch: 2, Loss: 0.1274656715089882
Epoch: 2, Loss: 0.15246144462638228
Epo